## 카카오 로컬 API 활용
-  도로명 주소 => 위도, 경도 좌표

In [23]:
import requests, json
import pandas as pd
from urllib.parse import quote

- API Key

In [24]:
with open('keys/카카오apikey.txt') as file:
    kakao_key = file.read()

- URL 만들기

In [25]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '경기도 부천시 길주로 16 (상동)'
url = f'{base_url}?query={quote(addr)}'

In [26]:
# Header: Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [27]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '경기 부천시 상동 549',
    'b_code': '4119010900',
    'h_code': '4119074400',
    'main_address_no': '549',
    'mountain_yn': 'N',
    'region_1depth_name': '경기',
    'region_2depth_name': '부천시',
    'region_3depth_h_name': '상동',
    'region_3depth_name': '상동',
    'sub_address_no': '',
    'x': '126.744149568416',
    'y': '37.506123727078'},
   'address_name': '경기 부천시 길주로 16',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '경기 부천시 길주로 16',
    'building_name': '상동호수공원',
    'main_building_no': '16',
    'region_1depth_name': '경기',
    'region_2depth_name': '부천시',
    'region_3depth_name': '상동',
    'road_name': '길주로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.744149568416',
    'y': '37.506123727078',
    'zone_no': '14592'},
   'x': '126.744149568416',
   'y': '37.506123727078'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [28]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [29]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.506123727078, 126.744149568416)

- 영등포 소재 지하철역 정보 완성하기

In [30]:
df = pd.read_csv('data/우리집주변.csv')
df

,이름,주소
0,상동호수공원,경기도 부천시 길주로 16 (상동)
1,부평역사박물관,인천광역시 부평구 굴포로 151 (삼산동)
2,부천체육관,경기도 부천시 석천로 293 (중동)
3,상동도서관,경기도 부천시 상이로 12 (상동)
4,부개도서관,인천광역시 부평구 부일로83번길 46 (부개동)


In [31]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [32]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,이름,주소,위도,경도
0,상동호수공원,경기도 부천시 길주로 16 (상동),37.506124,126.744150
1,부평역사박물관,인천광역시 부평구 굴포로 151 (삼산동),37.512142,126.737908
2,부천체육관,경기도 부천시 석천로 293 (중동),37.512757,126.763911
3,상동도서관,경기도 부천시 상이로 12 (상동),37.490140,126.744751
4,부개도서관,인천광역시 부평구 부일로83번길 46 (부개동),37.491021,126.739510


In [33]:
df.to_csv('data/우리집주변2.csv', index=False)
pd.read_csv('data/우리집주변2.csv')

,이름,주소,위도,경도
0,상동호수공원,경기도 부천시 길주로 16 (상동),37.506124,126.744150
1,부평역사박물관,인천광역시 부평구 굴포로 151 (삼산동),37.512142,126.737908
2,부천체육관,경기도 부천시 석천로 293 (중동),37.512757,126.763911
3,상동도서관,경기도 부천시 상이로 12 (상동),37.490140,126.744751
4,부개도서관,인천광역시 부평구 부일로83번길 46 (부개동),37.491021,126.739510
